In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from datetime import datetime
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback, TrainerState, TrainerControl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import torch
import numpy as np
import os
import warnings
import mlflow
import mlflow.pytorch

# Set the MLflow tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Suppress specific warnings
warnings.filterwarnings("ignore", message="Some weights of BertForSequenceClassification were not initialized from the model")
warnings.filterwarnings("ignore", message="You should probably TRAIN this model on a down-stream task")

def get_model_size(model):
    """
    Get the size of the model in memory.
    """
    torch.save(model.state_dict(), 'temp_model.pth')
    model_size = os.path.getsize('temp_model.pth')  # Size of the serialized model in bytes
    os.remove('temp_model.pth')  # Remove the temporary file
    return model_size

def calculate_unable_params(model, optimizer):
    """
    Calculate the number of parameters that are unable to be updated during training.
    """
    trainable_params = set(p for p in model.parameters() if p.requires_grad)
    trainable_param_ids = set(id(p) for p in trainable_params)
    optimizer_param_ids = set(id(p) for group in optimizer.param_groups for p in group["params"])
    unable_params_ids = trainable_param_ids - optimizer_param_ids
    unable_params = sum(p.numel() for p in trainable_params if id(p) in unable_params_ids)
    return unable_params

class LoggingCallback(TrainerCallback):
    def on_log(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        if state.log_history:
            log_history = state.log_history[-1]
            if 'loss' in log_history:
                mlflow.log_metric("train_loss", log_history['loss'], step=state.global_step)

def run_experiment(layer_name, layers_to_skip, num_samples, batch_size, epochs, experiment_name, dataset_path="customer_support_tickets.csv"):
    print(f"\nRunning experiment: {layer_name}")
    
    # Create a unique output directory
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f'./results_finetuned_{current_time}'
    os.makedirs(output_dir)
    
    # Load the dataset
    print("Loading dataset...")
    data = pd.read_csv(dataset_path)
    data = data.dropna(subset=['Ticket Description'])
    data = data[['Ticket Description', 'Ticket Type']]
    
    # Sample data if not using the full dataset
    if num_samples == "full":
        data_subset = data
    else:
        data_subset = data.sample(n=int(num_samples), random_state=42)
    
    # Split the data into train and test sets
    print("Splitting data into train and test sets...")
    train_texts, test_texts, train_labels, test_labels = train_test_split(data_subset['Ticket Description'], data_subset['Ticket Type'], test_size=0.2, random_state=42)
    
    # Encode labels
    print("Encoding labels...")
    label_encoder = LabelEncoder()
    train_labels_encoded = label_encoder.fit_transform(train_labels)
    test_labels_encoded = label_encoder.transform(test_labels)

    # Tokenization
    print("Tokenizing data...")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
    test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

    # Convert encodings to a format suitable for the trainer
    print("Converting encodings to PyTorch Dataset format...")
    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    train_dataset = MyDataset(train_encodings, train_labels_encoded)
    eval_dataset = MyDataset(test_encodings, test_labels_encoded)

    # Define BERT model
    print("Loading BERT model...")
    model_finetuned = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

    # Fine-tune by skipping intermediate layers
    if layers_to_skip == 'full':
        print("Not skipping any layers (training with all layers)...")
        model_pretrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
    else:    
        print("Skipping specified layers...")
        encoder_layers = model_finetuned.bert.encoder.layer
        model_finetuned.bert.encoder.layer = torch.nn.ModuleList([layer for i, layer in enumerate(encoder_layers) if i in layers_to_skip])

        
    # Define training arguments
    print("Defining training arguments...")
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy='epoch',
        logging_dir=f'./logs_finetuned_{current_time}',
        logging_steps=50,  # Log every 50 steps
        num_train_epochs=epochs,
        learning_rate=2e-5,
        save_strategy='epoch',
        load_best_model_at_end=True
    )

    # Define trainer
    print("Initializing trainer...")
    trainer = Trainer(
        model=model_finetuned,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        callbacks=[LoggingCallback()]
    )

    # Print the current tracking URI
    tracking_uri = mlflow.get_tracking_uri()
    print("Current tracking URI: ", tracking_uri)

    # Set the experiment name to be unique for the entire run
    mlflow.set_experiment(experiment_name)

    # Start an MLflow run with the current time in the name
    with mlflow.start_run(run_name=f"{layer_name}_{batch_size}_{epochs}_{current_time}"):
        mlflow.log_param("layer_name", layer_name)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("layers_to_skip", layers_to_skip)

        # Train and evaluate model
        print("Training and evaluating model...")
        start_time = time.time()
        trainer.train()
        training_time = time.time() - start_time

        # Predict test data
        print("Predicting test data...")
        start_time = time.time()
        preds = trainer.predict(eval_dataset)
        prediction_time = time.time() - start_time

        # Calculate metrics
        print("Calculating metrics...")
        accuracy = accuracy_score(test_labels_encoded, preds.predictions.argmax(axis=1))
        precision = precision_score(test_labels_encoded, preds.predictions.argmax(axis=1), average='weighted')
        recall = recall_score(test_labels_encoded, preds.predictions.argmax(axis=1), average='weighted')
        f1 = f1_score(test_labels_encoded, preds.predictions.argmax(axis=1), average='weighted')

        # Additional metrics for resource and efficiency comparison
        model_size = get_model_size(model_finetuned)
        num_params = sum(p.numel() for p in model_finetuned.parameters() if p.requires_grad)

        # Log metrics to MLflow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1", f1)
        mlflow.log_metric("training_time", training_time)
        mlflow.log_metric("prediction_time", prediction_time)
        mlflow.log_metric("model_size", model_size)
        mlflow.log_metric("num_params", num_params)

        # Save the model
        mlflow.pytorch.log_model(model_finetuned, "model")

        # Print results
        print("Results for Layer:", layer_name)
        results_df = pd.DataFrame({
            'Layer Name': [layer_name], 
            'Batch Size': [batch_size],
            'Epochs': [epochs],
            'Accuracy': [accuracy], 
            'Precision': [precision], 
            'Recall': [recall], 
            'F1': [f1], 
            'Training Time': [training_time],
            'Prediction Time': [prediction_time],
            'Model Size': [model_size],
            'Number of Parameters': [num_params],
        })
        display(results_df)

    return results_df




2024-07-14 12:26:40.500471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mlflow.get_tracking_uri()

'http://127.0.0.1:5000'

In [ ]:
# Example usage:
layers_to_skip = [
    np.random.choice(range(12), 6, replace=False),  # Random skipping
    np.random.choice(range(12), 6, replace=False),
    np.random.choice(range(12), 6, replace=False),
    [i for i in range(1, 12, 2)],  # Skipping odd layers
    [i for i in range(0, 12, 2)],  # Skipping even layers
    [1, 2, 3, 4, 5, 6, 7, 8],
    [5, 6, 7, 8, 9, 10, 11],
    []  # No skipping (full pre-trained)
]

num_samples = 'full'  # Specify the number of samples, or "full" for the full dataset
batch_sizes = [4,8,12]  # Example batch sizes
epoch_list = [3,4,5]  # Example numbers of epochs

# Set the experiment name
experiment_name = "BERT_Layer_Skipping_Experiment_Final_Thesis"

results_dfs = []
display(layers_to_skip)
sorted_layers_to_skip = [sorted(layer.tolist()) if isinstance(layer, np.ndarray) else layer for layer in layers_to_skip]
display(sorted_layers_to_skip)

# Run experiments for each configuration
for batch_size in batch_sizes:
    for epochs in epoch_list:
        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        print(f"Processing BATCHSIZE : {batch_size}  , EPOCHSIZE: {epochs}")
        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        for layers in sorted_layers_to_skip:
            if len(layers) == 0:
                name = "Full_Pretrained"
                layers = 'full'
            else:
                name = "Layer_" + "_".join([str(layer) for layer in layers]) + "_Skipped"
            results_df = run_experiment(name, layers, num_samples, batch_size, epochs, experiment_name)
            results_dfs.append(results_df)

# Combine results into a single DataFrame
combined_results = pd.concat(results_dfs, ignore_index=True)
print("\nCombined Results:")
display(combined_results)

[array([9, 3, 7, 4, 8, 5]),
 array([ 2,  5,  3,  0, 11,  6]),
 array([5, 6, 9, 8, 0, 3]),
 [1, 3, 5, 7, 9, 11],
 [0, 2, 4, 6, 8, 10],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [5, 6, 7, 8, 9, 10, 11],
 []]

[[3, 4, 5, 7, 8, 9],
 [0, 2, 3, 5, 6, 11],
 [0, 3, 5, 6, 8, 9],
 [1, 3, 5, 7, 9, 11],
 [0, 2, 4, 6, 8, 10],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [5, 6, 7, 8, 9, 10, 11],
 []]

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Processing BATCHSIZE : 4  , EPOCHSIZE: 3
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

Running experiment: Layer_3_4_5_7_8_9_Skipped
Loading dataset...
Splitting data into train and test sets...
Encoding labels...
Tokenizing data...
Converting encodings to PyTorch Dataset format...
Loading BERT model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/monu/opt/anaconda3/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Skipping specified layers...
Defining training arguments...
Initializing trainer...


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Not skipping any layers (training with all layers)...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/monu/opt/anaconda3/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Defining training arguments...
Initializing trainer...
Current tracking URI:  http://127.0.0.1:5000
Training and evaluating model...


Epoch,Training Loss,Validation Loss
1,1.616400,1.610187
2,1.618600,1.611489
3,1.613800,1.612135
4,1.603300,1.621262
5,1.564700,1.665331
